In [1]:
from AOC_utils import get_day
import numpy as np

day = 14
input_data = get_day(day)

# print out first few lines to get a feel for the data
input_data[:7]

Downloaded day 14 input


['#..O...O...O....##O#O#O..O.#...O..O..#O.#O.#.O....##.OOOO....O.O.OO..O##..O..O....#O#....#..O.#.#.O.',
 'OO.....O.O...##..#.#.......#...#.O.#.#..#...#...#....##O......#O#........#...O......O.#......#O.O...',
 '.O..##.OO.OO#OO.......O..O.#...#O.O...O.O.#..OO.#...O.OO.#.O.#....O.O..O.O#O...O...#.O..O...#......O',
 '...........#...#.O..O.#.O.#..O#.O.OO.....##O..##..OO..O..#OO..O...#.#..............OOO.......#...O..',
 '..##.O#.....OO.O#O.#.##.....O...#...#..O#O#.....#..O..O................OO..........#....#..O#....#..',
 '..OO.....O..#.....O...#O.O.O...#....##...#....O.OO#.#..#O..O.....O.O...#.O...OO..O.O#O.O#OO#..O..#..',
 'O...O....#.##O........................O....#...#....#.O#..O.....O..O.##O#...O.#..O..OO...O#O.#....O.']

In [2]:
example = '''
O....#....
O.OO#....#
.....##...
OO.#O....O
.O.....O#.
O.#..O.#.#
..O..#O..O
.......O..
#....###..
#OO..#....
'''.split('\n')[1:-1]

In [ ]:
len(input_data)

In [136]:
def parse_data(data):

    data = np.array([list(line) for line in data])

    print(data)
    print()

    def slide_rocks_north(data):
        slideable_rocks = np.argwhere(data == 'O').tolist()
        
        for r in range(len(slideable_rocks)):
            rock = slideable_rocks[r]
            i, j = rock
            if i == 0:
                continue
            # get slice of all spaces above rock
            above = data[:i, j]
            # move rock up until it hits a wall or another rock
            while i > 0 and above[-1] == '.':
                i -= 1
                above = data[:i, j]
            # if i is different, rock has moved
            if i != rock[0]:
                # place rock in new location
                data[i, j] = 'O'
                # remove rock from old location
                data[rock[0], rock[1]] = '.'
        return data
    
    def slide_rocks_west(data):
        slideable_rocks = np.argwhere(data == 'O').tolist()
        
        for r in range(len(slideable_rocks)):
            rock = slideable_rocks[r]
            i, j = rock
            if j == 0:
                continue
            # get slice of all spaces above rock
            left = data[i, :j]
            # move rock up until it hits a wall or another rock
            while j > 0 and left[-1] == '.':
                j -= 1
                left = data[i, :j]
            # if j is different, rock has moved
            if j != rock[1]:
                # place rock in new location
                data[i, j] = 'O'
                # remove rock from old location
                data[rock[0], rock[1]] = '.'
        return data
    
    def slide_rocks(data, direction):

        if direction == 'north':
            data = slide_rocks_north(data)
    
        elif direction == 'west':
            data = slide_rocks_west(data)

        elif direction == 'south':
            # reverse order of rocks so they slide from bottom up
            data = slide_rocks(data[::-1], 'north')
            data = data[::-1]
                
        elif direction == 'east':
            # reverse order of rocks so they slide from bottom up
            data = slide_rocks(data[:, ::-1], 'west')
            data = data[:, ::-1]
        
        return data
    

    def get_total_load(data):
        slideable_rocks = np.argwhere(data == 'O')[:, 0].tolist() # only need y axis
        total_load = 0
        for rock in slideable_rocks:
            total_load += len(data) - rock
        return total_load   

    data_sn = slide_rocks_north(data)
    print(data_sn)
    print()
    
    total = get_total_load(data_sn)

    def cycle(data):
        data = slide_rocks(data, 'north')
        data = slide_rocks(data, 'west')
        data = slide_rocks(data, 'south')
        data = slide_rocks(data, 'east')
        return data

    data2 = data.copy()

    dict_cache = {}
    def find_cycle(data):
        i = 1
        while i < 1000000000:
            data = cycle(data)
            string_data = "".join(data.flatten())

            if string_data in dict_cache:
                return i, dict_cache[string_data], data
            dict_cache[string_data] = i
            i += 1

    cycle_end, cycle_start, data2 = find_cycle(data2)
    cycle_length = cycle_end - cycle_start
    print(cycle_end, cycle_start, cycle_length)

    # use cycle length to find the state of the data after 1 billion cycles    
    start = cycle_end
    while start < 1000000000:
        start += cycle_length
    start -= cycle_length

    while start < 1000000000:
        data2 = cycle(data2)
        start += 1

    # def get_state_at_n(n, cycle_start, cycle_length, cache):
    #     if n <= cycle_start + cycle_length:
    #         return [k for k, v in cache.items() if v == n][0]
    #     n = n - cycle_start
    #     n = n % cycle_length
    #     return [k for k, v in cache.items() if v == n][0]

    print(data2)

    total2 = get_total_load(data2)

    

    print("part 1:", total)
    print("part 2:", total2)

In [132]:
parse_data(input_data)

[['#' '.' '.' ... '.' 'O' '.']
 ['O' 'O' '.' ... '.' '.' '.']
 ['.' 'O' '.' ... '.' '.' 'O']
 ...
 ['O' '#' '.' ... '.' '.' '.']
 ['#' '.' 'O' ... '.' '#' 'O']
 ['.' '#' 'O' ... '.' '.' '.']]

[['#' 'O' '.' ... 'O' 'O' 'O']
 ['O' 'O' '.' ... '.' 'O' 'O']
 ['O' 'O' '.' ... '.' '.' 'O']
 ...
 ['.' '#' '.' ... '.' '.' 'O']
 ['#' '.' '.' ... '.' '#' '.']
 ['.' '#' '.' ... '.' '.' '.']]

153 139 14
[['#' '.' '.' ... '.' '.' '.']
 ['.' '.' '.' ... '.' '.' 'O']
 ['O' 'O' 'O' ... '.' 'O' 'O']
 ...
 ['O' '#' '.' ... '.' '.' 'O']
 ['#' '.' 'O' ... '.' '#' '.']
 ['.' '#' '.' ... '.' '.' '.']]
part 1: 108889
part 2: 104671


In [ ]:
104663
104667
101446

In [134]:
parse_data(example)

[['O' '.' '.' '.' '.' '#' '.' '.' '.' '.']
 ['O' '.' 'O' 'O' '#' '.' '.' '.' '.' '#']
 ['.' '.' '.' '.' '.' '#' '#' '.' '.' '.']
 ['O' 'O' '.' '#' 'O' '.' '.' '.' '.' 'O']
 ['.' 'O' '.' '.' '.' '.' '.' 'O' '#' '.']
 ['O' '.' '#' '.' '.' 'O' '.' '#' '.' '#']
 ['.' '.' 'O' '.' '.' '#' 'O' '.' '.' 'O']
 ['.' '.' '.' '.' '.' '.' '.' 'O' '.' '.']
 ['#' '.' '.' '.' '.' '#' '#' '#' '.' '.']
 ['#' 'O' 'O' '.' '.' '#' '.' '.' '.' '.']]

[['O' 'O' 'O' 'O' '.' '#' '.' 'O' '.' '.']
 ['O' 'O' '.' '.' '#' '.' '.' '.' '.' '#']
 ['O' 'O' '.' '.' 'O' '#' '#' '.' '.' 'O']
 ['O' '.' '.' '#' '.' 'O' 'O' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '#' '.']
 ['.' '.' '#' '.' '.' '.' '.' '#' '.' '#']
 ['.' '.' 'O' '.' '.' '#' '.' 'O' '.' 'O']
 ['.' '.' 'O' '.' '.' '.' '.' '.' '.' '.']
 ['#' '.' '.' '.' '.' '#' '#' '#' '.' '.']
 ['#' '.' '.' '.' '.' '#' '.' '.' '.' '.']]

10 3 7
[['.' '.' '.' '.' '.' '#' '.' '.' '.' '.']
 ['.' '.' '.' '.' '#' '.' '.' '.' 'O' '#']
 ['.' '.' '.' '.' '.' '#' '#' '.' '.' '.']
